In [20]:
import requests
import json
import math
import pandas as pd
from bs4 import BeautifulSoup

In [21]:
## Configurations

# Trustpilot review page
url = 'https://uk.trustpilot.com/review/www.johnlewis.com'

# Data file to save to
final_list=[]#final list to be the df
save_datafile = 'trust_reviews.csv'

# Trustpilot default 
results_per_page = 20 
total_pages = 1

print('Scraper set for ' + url + ' - saving result to ' + save_datafile)

Scraper set for https://uk.trustpilot.com/review/www.johnlewis.com - saving result to trust_reviews.csv


In [22]:
## Count amount of pages to scrape

# Get page
r=requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

# Total amount of ratings
rating_count = soup.find('span',class_='headline__review-count')
rating_count = int(rating_count.text.replace(',',''))

# Throttling to avoid spamming page with requests
# With sleepTime seconds between every page request
throttle = False
sleep_time = 2

# Total pages to scrape
total_pages = math.ceil(rating_count / results_per_page)

print('Found total of ' + str(total_pages) + ' pages to scrape')

Found total of 485 pages to scrape


In [23]:
## Run Scraper
for pg in range(1, total_pages + 1):
    pg = url + '?page=' + str(pg)
    r=requests.get(pg)
    soup = BeautifulSoup(r.text, 'lxml')
        
    for paragraph in soup.find_all('section',class_='review__content'):
        title_section=paragraph.find('h2',class_='review-content__title')   
        
        if title_section:
            title = title_section.find('a').text.strip()
        else:
            title = ''
            
        content=paragraph.find('p',class_='review-content__text')
        
        if content:
            content = content.text.strip()
            datedata= json.loads(paragraph.find('div',class_='review-content-header__dates').text)
            date=datedata['publishedDate'].split('T')[0]
            rating_class=paragraph.find('div',class_='star-rating')

            rating=rating_class.find('img')['alt'][0]   

            final_list.append([title,content,date,rating])

    if(throttle): 
        time.sleep(sleep_time)
        
df = pd.DataFrame(final_list,columns=['Title','Content','Date','Rating'])
print(df)

                                                Title  \
0     Unsatisfactory Customer Service from John Lewis   
1              Be careful about using Click & Collect   
2               I am still waiting to hear from John…   
3          John Lewis Customer Service! doesn't exist   
4   Warranty worse than useless…  on electronic go...   
5        FOLLOW UP TO WARNING ABOUT APPROVED SUPPLIER   
6                          Excellent customer service   
7                              What Customer service!   
8                       Excellent after sales service   
9             Terrible customer service at Newcastle…   
10                      Over promised under delivered   
11                               Hi again John Lewis.   
12                  Faulty television, refund refused   
13                   Yet again JL let themselves down   
14                      Made to feel like a criminal!   
15                              Disappointing service   
16                             

In [24]:
## Save dataframe to csv
df.to_csv(save_datafile, encoding='utf-8', index=False)